In [1]:
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt
import seaborn as sns

from aqmsp_data.data import load_caaqm

## Checks

In [2]:
data_22 = (
    load_caaqm(years=2022, months=1, variables=["PM2.5"])
    .to_dataframe()
    .reset_index()[["station", "latitude", "longitude", "PM2.5"]]
)
data_23 = (
    load_caaqm(years=2023, months=1, variables=["PM2.5"])
    .to_dataframe()
    .reset_index()[["station", "latitude", "longitude", "PM2.5"]]
)
data_23.shape, data_22.shape


((29800, 4), (29760, 4))

In [3]:
stations = data_22.dropna()["station"].value_counts().index.values.tolist()
######### Warning #########
# Always drop "Pusa, Delhi - IMD" since it has the wrong coordinates
###########################
stations.remove("Pusa, Delhi - IMD")
len(stations)


38

In [4]:
stations_df = (
    data_22[data_22.station.isin(stations)]
    .drop_duplicates("station")
    .drop("PM2.5", axis=1)
)
print(stations_df.shape)
stations_df.head()


(38, 3)


,station,latitude,longitude
0,"Alipur, Delhi - DPCC",28.815329,77.153010
744,"Anand Vihar, Delhi - DPCC",28.647622,77.315809
1488,"Ashok Vihar, Delhi - DPCC",28.695381,77.181665
2232,"Aya Nagar, Delhi - IMD",28.470691,77.109936
2976,"Bawana, Delhi - DPCC",28.776200,77.051074


In [5]:
stations_df.set_index("station").to_xarray().to_netcdf("../data/caaqm/stations.nc")